In [1]:
%load_ext autotime

time: 0 ns


In [2]:
import torch
import torch.nn as nn
import torchvision.models as models
import torch.nn.functional as F
import numpy as np

time: 4.69 s


## 依照指示取出模型特定層的資訊

In [3]:
# Define model
class TheModelClass(nn.Module):
    def __init__(self):
        super(TheModelClass, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16* 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 16 * 5 * 5)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

# Initialize model
model = TheModelClass()


time: 47 ms


### 取出 self.pool層兩次的輸出，包含：
* x = self.pool(F.relu(self.conv1(x)))
* x = self.pool(F.relu(self.conv2(x)))

In [7]:
model.pool.regi

MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)

time: 0 ns


In [8]:
outputs= []
def layer1_hook(module, input_, output):
    outputs.append(output)
    
model.pool.register_forward_hook(layer1_hook)

time: 0 ns


In [9]:
input_ = torch.randn(1, 3, 32, 32)
output = model(input_)

time: 47 ms


In [10]:
outputs

[tensor([[[[0.1291, 0.7673, 0.2730,  ..., 0.5543, 0.6246, 0.9026],
           [0.4778, 0.7482, 0.8064,  ..., 0.2080, 0.5661, 1.0232],
           [0.0000, 1.0197, 0.7304,  ..., 0.7775, 0.0479, 1.0634],
           ...,
           [0.3543, 0.6955, 0.6641,  ..., 0.1598, 1.0707, 0.8102],
           [0.8117, 0.6378, 0.6097,  ..., 0.8016, 0.3826, 0.6265],
           [0.0000, 0.0000, 0.2653,  ..., 0.4369, 1.3566, 0.4295]],
 
          [[0.0000, 0.6799, 0.4552,  ..., 0.4724, 1.0408, 0.4997],
           [0.5225, 0.2341, 0.2099,  ..., 0.7007, 0.5945, 0.4289],
           [0.3145, 0.4256, 0.7829,  ..., 0.5420, 0.2392, 0.0000],
           ...,
           [0.5516, 1.1926, 0.0414,  ..., 0.5872, 0.1823, 1.0445],
           [0.5672, 0.2747, 1.3825,  ..., 1.5253, 0.0000, 0.2044],
           [1.1402, 0.0779, 0.0000,  ..., 0.5541, 0.6493, 0.5141]],
 
          [[0.1573, 0.5502, 0.5729,  ..., 0.6183, 0.3857, 1.5685],
           [1.4748, 0.5035, 0.8282,  ..., 0.0000, 2.1505, 0.5575],
           [0.8099, 1.10

time: 47 ms


In [11]:
# 與解答相符
print(outputs[0].shape)
print(outputs[1].shape)

torch.Size([1, 6, 14, 14])
torch.Size([1, 16, 5, 5])
time: 0 ns


In [67]:
print(outputs[0].shape)
print(outputs[1].shape)

torch.Size([1, 6, 14, 14])
torch.Size([1, 16, 5, 5])


## 加入自定義 initialization fuction

#### 對所有Conv2D層使用自定義initialization function
* weight : nn.init.kaiming_normal_
* bias : 全部輸入1

In [12]:
model

TheModelClass(
  (conv1): Conv2d(3, 6, kernel_size=(5, 5), stride=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=400, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)

time: 0 ns


In [13]:
from torch.nn import init

def weights_init(m):
    #classname = m.__class__.__name__
    if isinstance(m, nn.Conv2d):
        #torch.nn.init.xavier_uniform_(m.weight)
        nn.init.kaiming_normal_(m.weight.data,
                                    a=0,
                                    mode='fan_out',
                                    nonlinearity='relu')
        if m.bias is not None:
            nn.init.ones_(m.bias)
            
model.apply(weights_init)

TheModelClass(
  (conv1): Conv2d(3, 6, kernel_size=(5, 5), stride=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=400, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)

time: 16 ms


In [14]:
for name, parameters in model.named_parameters():
    if ('conv' in name) and ('bias' in name):
        print(name, parameters)
        print('\n')

conv1.bias Parameter containing:
tensor([1., 1., 1., 1., 1., 1.], requires_grad=True)


conv2.bias Parameter containing:
tensor([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
       requires_grad=True)


time: 0 ns


In [71]:
from torch.nn import init

def weights_init(m):
    #classname = m.__class__.__name__
    if isinstance(m, nn.Conv2d):
        #torch.nn.init.xavier_uniform_(m.weight)
        nn.init.kaiming_normal_(m.weight.data,
                                    a=0,
                                    mode='fan_out',
                                    nonlinearity='relu')
        if m.bias is not None:
            nn.init.ones_(m.bias)
            
model.apply(weights_init)

TheModelClass(
  (conv1): Conv2d(3, 6, kernel_size=(5, 5), stride=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=400, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)

#### 查看 conv層的bias是否皆為1

In [80]:
for name, parameters in model.named_parameters():
    if ('conv' in name) and ('bias' in name):
        print(name, parameters)
        print('\n')

conv1.bias Parameter containing:
tensor([1., 1., 1., 1., 1., 1.], requires_grad=True)


conv2.bias Parameter containing:
tensor([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
       requires_grad=True)


